This is a notebook I've created for last year's HuBMAP competition. As I'm sure the topic will come up again, I've created this fork to verify that it also works on the current 2022 competition.
I wrote these routines from scratch, they are an order of magnitude faster than all others I've seen up to now and are also 100% correct.

These routines are
* memory-efficient
* fast
* definitely correct on a pixel basis for the specific format used in this competition.

Some other routines I've found were off 1 pixel (not that it matters, but aren't we all a bit nitpicky in this line of work ;)) or had other limitations.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tifffile as tiff

In [ ]:
def rle2mask(rle, mask_shape):
    ''' takes a space-delimited RLE string in column-first order
    and turns it into a 2d boolean numpy array of shape mask_shape '''
    
    mask = np.zeros(np.prod(mask_shape), dtype=bool) # 1d mask array
    rle = np.array(rle.split()).astype(int) # rle values to ints
    starts = rle[::2]
    lengths = rle[1::2]
    for s, l in zip(starts, lengths):
        mask[s:s+l] = True
    return mask.reshape(np.flip(mask_shape)).T # flip because of column-first order


def mask2rle(mask):
    ''' takes a 2d boolean numpy array and turns it into a space-delimited RLE string '''
    
    mask = mask.T.reshape(-1) # make 1D, column-first
    mask = np.pad(mask, 1) # make sure that the 1d mask starts and ends with a 0
    starts = np.nonzero((~mask[:-1]) & mask[1:])[0] # start points
    ends = np.nonzero(mask[:-1] & (~mask[1:]))[0] # end points
    rle = np.empty(2 * starts.size, dtype=int) # interlacing...
    rle[0::2] = starts # ...starts...
    rle[1::2] = ends - starts # ...and lengths
    rle = ' '.join([ str(elem) for elem in rle ]) # turn into space-separated string
    return rle

Let's check with the original train RLEs:

In [ ]:
df_enc = pd.read_csv('../input/hubmap-organ-segmentation/train.csv')
df_enc

We'll take the first image:

In [ ]:
enc_original = df_enc.loc[0,"rle"]
enc_original[:1000]

In [ ]:
tiff_shape = tiff.TiffFile('../input/hubmap-organ-segmentation/train_images/10044.tiff').pages[0].shape[:2]
tiff_shape

Fast decoding:

In [ ]:
%%time
mask = rle2mask(enc_original, tiff_shape)

Mask looks ok *and is in the correct numpy orientation*:

In [ ]:
plt.imshow(mask[::10,::10]);

Fast encoding, no RAM was harmed in the process...

In [ ]:
%%time
enc_reencoded = mask2rle(mask)

And the final check:

In [ ]:
enc_reencoded[:1000]

In [ ]:
enc_original == enc_reencoded